In [1]:
from sklearn.cross_validation import KFold

In [3]:
KFold?

In [ ]:
import numpy as np
from PIL import Image
import os
import pandas as pd

Open root directory:

In [ ]:
root = '/Users/brian/Documents/datasets/lisa_set'

Create a list of the-sub directories that have images:

In [ ]:
image_dirs = [str(x) for x in os.listdir(root) if x[-1].isdigit()]

Get all further sub-directories with images in them:

In [ ]:
image_paths = []
for direct in image_dirs:
    for subdir in os.listdir(root+'/'+direct)[:]:
        if subdir[0].isalpha():
            image_paths.append(root+'/'+direct+'/'+subdir+'/')

In [ ]:
bounding = 'frameAnnotations.csv' # String name for the annotations

In [ ]:
for i in range(len(image_paths)):
    make_images(i)
    print "Finished set: {}".format(i)

In [ ]:
def make_images(current_ind):
    print "Starting set: " + str(current_ind)
    current = image_paths[current_ind]
    images = [x for x in os.listdir(current) if '.png' in x]
    image_annot = pd.read_csv(current+bounding, delimiter=';')

    save_path = r'/Users/brian/Google Drive/Galvanize/course/capstone/signclassification/data/fullset/set_{}'.format(current_ind) 
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        print "Creating path: " + save_path
    with open('data/fullset/set_{}/path.txt'.format(current_ind), 'w') as f:
        f.write(image_paths[i])

    image_bounds_1 = {}
    for row in range(len(image_annot)):
        image_bounds_1[image_annot.loc[row][0]] = (image_annot.loc[row][1],[float(x) for x in str(image_annot.loc[row][2:6]).split() if x.isdigit()])

    saved_images = []
    for im in image_bounds_1.keys():
        img = Image.open(current+im)
        img2 = img.crop(tuple(image_bounds_1[im][1]))
        longer_side = max(img2.size)
        horizontal_padding = (longer_side - img2.size[0]) / 2
        vertical_padding = (longer_side - img2.size[1]) / 2
        img3 = img2.crop(
        (
            -horizontal_padding,
            -vertical_padding,
            img2.size[0] + horizontal_padding,
            img2.size[1] + vertical_padding
        )
        )
        if img3.size[0] > img3.size[1]:
            img4 = img3.crop((0,0,img3.size[1],img3.size[1]))
        elif img3.size[0] < img3.size[1]:
            img4 = img3.crop((0,0,img3.size[0],img3.size[0]))
        else:
            img4 = img3
        image_str = '{}{}'.format(image_bounds_1[im][0], str(img4.size))
        x = 0
        while image_str + str(x) in saved_images:
            x += 1
        image_str += str(x)
        saved_images.append(image_str)
        img4.save("./data/fullset/set_{}/{}.jpg".format(current_ind, image_str))
    